# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium

# Import API key
from api_keys import geoapify_key

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
# Create a base map
map_city = folium.Map(location=[city_data_df['Lat'].mean(), city_data_df['Lng'].mean()], 
                      zoom_start=5, control_scale=True)

# Add points to the map for each city with the size based on humidity
for index, row in city_data_df.iterrows():
    folium.CircleMarker(location=[row['Lat'], row['Lng']], 
                        radius=row['Humidity'],  # Size of the point based on humidity
                        color='blue', 
                        fill=True, 
                        fill_color='blue', 
                        fill_opacity=0.6, 
                        tooltip=f"Humidity: {row['Humidity']}").add_to(map_city)

# Display the map
map_city


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit the specified weather conditions
ideal_cities_df = city_data_df[(city_data_df['Max Temp'] > 21) & 
                               (city_data_df['Max Temp'] < 27) & 
                               (city_data_df['Wind Speed'] < 4.5) & 
                               (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values (if necessary)
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
print(ideal_cities_df.head())


### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000  # You can adjust the radius (in meters) based on your preference
params = {
    "type": "hotel",
    "limit": 1,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"point({lng},{lat})"
    params["bias"] = f"point({lng},{lat})"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print(hotel_df)


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
# Set the center of the map to the average latitude and longitude of the cities in hotel_df
map_center = [hotel_df['Lat'].mean(), hotel_df['Lng'].mean()]

# Configure the map plot
map_hotels = folium.Map(location=map_center, zoom_start=5, control_scale=True)

# Add markers for each hotel in hotel_df
for index, row in hotel_df.iterrows():
    hotel_name = row['Hotel Name']
    city_name = row['City']
    country_name = row['Country']
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Create a popup message for the marker
    popup_msg = f"{hotel_name}, {city_name}, {country_name}"
    
    # Add a marker to the map
    folium.Marker(
        location=[latitude, longitude],
        popup=popup_msg,
        icon=folium.Icon(color='blue', icon='hotel')
    ).add_to(map_hotels)

# Display the map
map_hotels
